# Reading and Writing Data in Text Format

*Parsing functions in pandas*

|Function|Description|
|--------|------------|
|read_csv|Load delimited data from a file, URL, or file-like object. Use comma as default delimiter|
|read_table|Load delimited data from a file, URL, or file-like object. Use tab('\t') as default delimiter|
|read_fwf|Read data in fixed-width column format (that is, no delimiters)
|read_clipboard|Version of read_table that reads data from the clipboard. Usefule for converting tables from web pages|
|read_html|||


- Indexing: can treat one or more columns as the returned DataFrame, and whether to get column names from the file, the user, or not at all.
- Type inference and data conversion: this includes the user-defined value conversions and custom list of missing value markers.
- Datetime parsing: includes combining capability, including combining data and time information spread over multiple columns into a single column in the result.
- Iterating: support for iterating over chunks of very large files
- Unclean data issues: skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

* [Download example data](https://github.com/wesm/pydata-book/tree/master/ch06)

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

In [ ]:
!cat ./ex1.csv     # For u*ix, for windows, use "type"

In [ ]:
df = pd.read_csv('./ex1.csv')
df

In [ ]:
pd.read_table('./ex1.csv', sep=',')

In [ ]:
!cat ./ex2.csv

In [ ]:
pd.read_csv('./ex2.csv', header=None)

In [ ]:
pd.read_csv('./ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

In [ ]:
# index -> 'message' column
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./ex2.csv', names=names, index_col='message')

In [ ]:
!cat ./csv_mindex.csv

In [ ]:
# hierachical index from multiple columns
parsed = pd.read_csv('./csv_mindex.csv', index_col=['key1', 'key2'])
parsed

In [ ]:
!cat ./ex3.txt

In [ ]:
result = pd.read_table('./ex3.txt')
result

In [ ]:
list(open('./ex3.txt'))

In [ ]:
result = pd.read_table('./ex3.txt', sep='\s+')
result   # one fewer column name? -> inners that the first columns should be the DataFrame's index

In [ ]:
!cat ./ex4.csv

In [ ]:
pd.read_csv('./ex4.csv', skiprows=[0, 2, 3])

In [ ]:
# "sentinel" value : NA, -1.#IND, NULL
!cat ./ex5.csv

In [ ]:
result = pd.read_csv('./ex5.csv')
result

In [ ]:
pd.isnull(result)

In [ ]:
# "na_values" option
result = pd.read_csv('./ex5.csv', na_values=['NULL'])
result

In [ ]:
# differnet NA setinels

In [ ]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

In [ ]:
pd.read_csv('./ex5.csv', na_values=sentinels)

*read_csv/read_table function arguments*

|Argument|Description|
|----------|---------------------|
|path||
|sep or delimiter||
|header||
|index_col||
|names||
|skiprows||
|na_values||
|comment||
|parse_dates|Fase by default|
|keep_date_col||
|converters|Dict containing column number of name mapping functions.|
|dayfirst||
|date_parser||
|nrows|Number of rows to read from befinning of file|
|iterator||
|chunksize|For iteration, size of file chunks|
|skip_footer||
|verbose||
|encoding||
|squeeze||
|thousands||

## Reading Text Files in Pieces

In [ ]:
result = pd.read_csv('./ex6.csv')
result

In [ ]:
# only read out a small number of rows
pd.read_csv('./ex6.csv', nrows=5)

In [ ]:
# chunksize
chunker = pd.read_csv('./ex6.csv', chunksize=1000)

In [ ]:
chunker

In [ ]:
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)

In [ ]:
tot[:10]

## Writing Data Out to Text Format

In [ ]:
data = pd.read_csv('./ex5.csv')
data

In [ ]:
data.to_csv('./out.csv')

In [ ]:
!cat ./out.csv

In [ ]:
# other delimiters
import sys
data.to_csv(sys.stdout, sep='|')

In [ ]:
# denote missing values by some other sentinel value
data.to_csv(sys.stdout, na_rep='NULL')

In [ ]:
# don't write row and column lables
data.to_csv(sys.stdout, index=False, header=False)

In [ ]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

In [ ]:
# Series
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts

In [ ]:
ts.to_csv('./tseries.csv')

In [ ]:
!cat ./tseries.csv

In [ ]:
Series.from_csv('./tseries.csv', parse_dates=True)

--------------------------------------------------
## Manually Working with Delimited Formats

In [ ]:
!cat ./ex7.csv

In [ ]:
# For any file with single-character delimiter, can use Python's built-in csv module
import csv
f = open('./ex7.csv')

In [ ]:
reader = csv.reader(f)

In [ ]:
for line in reader:
    print(line)

In [ ]:
lines = list(csv.reader(open('./ex7.csv')))

In [ ]:
header, values = lines[0], lines[1:]

In [ ]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

In [ ]:
# csv.Dialect
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
    
reader = csv.reader(f, dialect=my_dialect)

*CSV dialect options*

|Argument|Description|
|---------|--------------|
|delimiter||
|lineterminator||
|quotechar||
|quoting||
|skipinitialspace||
|doublequote||
|escapechar||

In [ ]:
# csv.writer
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

## JSON Data
JavsScript Object Notation : become one of the standard formats for sending data by HTTP request between web browsers and other applications.

In [ ]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

- Basic types : objects(dicts), arrays(lists), strings, numbers, booleans, and nulls
- All of the keys in an object must be strings
- Disallowing trailing commmas at the end of lists

In [ ]:
import json

In [ ]:
# JSON strong -> Python form
result = json.loads(obj)
result

In [ ]:
# Python object -> JSON
asjson = json.dumps(result)

In [ ]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])

In [ ]:
siblings

- to_json()
- read_json()